# Splicing the mind: Master's Thesis
Preprocessing pipeline for DEXSeq analysis between schizophrenia patients and contols in the nucleus accumbens

This notebook is for querying the OmicSoft DiseaseLAnd data source and preparing data to be in an adequate format for DEXSeq analysis in R.

In [ ]:
# Import necessary libraries
from opi import *

datastore = OPI()

import pandas as pd
import math

# Always display only 5 rows in this NB
pd.set_option('display.max_rows', 5)

### Querying the data

#### Metadata extraction

In [ ]:
# Query metadata of schizophrenia studies from the nucleus accumbens tissue and white ethnicity
sql = """ 
WITH valid_samples AS (
  SELECT DISTINCT sample_index
  FROM human_disease_b38_gc33.exon_data
)
SELECT 
  s.sample_index,
  s.age_summary,
  s.disease_state,
  s.ethnicity,
  s.gender,
  s.tissue,
  s.subject_id,
  p.disease,
  p.title
FROM human_disease_b38_gc33.samples s
JOIN human_disease_b38_gc33.projects p 
  ON s.project_id = p.project_id
WHERE s.disease_state IN 'schizophrenia'           # eaarlier: ('schizophrenia', 'schizoaffective disorder')
  AND s.tissue = 'nucleus accumbens'
  AND s.ethnicity = 'White'
  AND s.sample_index IN (SELECT sample_index FROM valid_samples)
  LIMIT 5
"""
sz_nacc_metadata = datastore.query(sql)
display(sz_nacc_metadata)

,sample_index,age_summary,disease_state,ethnicity,gender,tissue,subject_id,disease,title
0,85772,31 years,schizophrenia,White,female,nucleus accumbens,GSE80655_X2353,mental disorder,RNA-sequencing of human post-mortem brain tissues
1,85966,31 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X4463,mental disorder,RNA-sequencing of human post-mortem brain tissues
2,85819,58 years,schizophrenia,White,female,nucleus accumbens,GSE80655_X4114,mental disorder,RNA-sequencing of human post-mortem brain tissues
3,86046,50 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X4404,mental disorder,RNA-sequencing of human post-mortem brain tissues
4,221007,44 years,schizophrenia,White,male,nucleus accumbens,GSE202537_933,psychotic disorder,Diurnal alterations in gene expression across ...
5,85907,36 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X3410,mental disorder,RNA-sequencing of human post-mortem brain tissues
6,85942,45 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X2950,mental disorder,RNA-sequencing of human post-mortem brain tissues
7,85908,24 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X4385,mental disorder,RNA-sequencing of human post-mortem brain tissues
8,220944,49 years,schizophrenia,White,male,nucleus accumbens,GSE202537_1088,psychotic disorder,Diurnal alterations in gene expression across ...
9,86033,50 years,schizophrenia,White,male,nucleus accumbens,GSE80655_X2384,mental disorder,RNA-sequencing of human post-mortem brain tissues


In [140]:
# Sanity check
sz_nacc_metadata["tissue"].value_counts()

tissue
nucleus accumbens    32
Name: count, dtype: Int64

In [141]:
sz_nacc_metadata["disease"].value_counts()

disease
mental disorder       20
psychotic disorder    12
Name: count, dtype: Int64

In [142]:
sz_nacc_metadata["title"].value_counts()

title
RNA-sequencing of human post-mortem brain tissues                                 20
Diurnal alterations in gene expression across striatal subregions in psychosis    12
Name: count, dtype: Int64

title
Diurnal alterations in gene expression across striatal subregions in psychosis    36
RNA-sequencing of human post-mortem brain tissues                                 22
Name: count, dtype: Int64

In [ ]:
# Query normal control metadata - NAcc, white
sql = """ 
WITH valid_samples AS (
  SELECT DISTINCT s.sample_index
  FROM human_disease_b38_gc33.samples s
  JOIN human_disease_b38_gc33.projects p 
    ON s.project_id = p.project_id
  JOIN human_disease_b38_gc33.exon_data ed
    ON s.sample_index = ed.sample_index
  WHERE s.disease_state IN ('normal control', 'disease control')
    AND s.tissue = 'nucleus accumbens'
    AND s.ethnicity = 'White'
    AND p.title IN (
         'RNA-sequencing of human post-mortem brain tissues', 
         'Diurnal alterations in gene expression across striatal subregions in psychosis'
    )
)
SELECT 
  s.sample_index,
  s.age_summary,
  s.disease_state,
  s.ethnicity,
  s.gender,
  s.tissue,
  p.disease,
  p.title
FROM human_disease_b38_gc33.samples s
JOIN human_disease_b38_gc33.projects p 
  ON s.project_id = p.project_id
WHERE s.sample_index IN (SELECT sample_index FROM valid_samples)
LIMIT 5
"""
normal_nacc_metadata = datastore.query(sql)
display(normal_nacc_metadata)

,sample_index,age_summary,disease_state,ethnicity,gender,tissue,disease,title
0,85954,63 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues
1,85953,32 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues
2,220954,36 years,disease control,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
3,220996,38 years,disease control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
4,220958,58 years,disease control,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
5,220979,45 years,disease control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
6,220940,23 years,disease control,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
7,220988,49 years,disease control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...
8,85836,66 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues
9,220942,49 years,disease control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...


In [ ]:
normal_nacc_metadata["title"].value_counts()

### Cleaning up the resulting dfs and merging them

In [189]:
# Extract numeric digits from the 'age_summary' column and convert to int
normal_nacc_metadata['age'] = normal_nacc_metadata['age_summary'].str.extract(r'(\d+)').astype(int)
sz_nacc_metadata["age"] = sz_nacc_metadata['age_summary'].str.extract(r'(\d+)').astype(int)

In [190]:
# Set disease control to normal control
normal_nacc_metadata["disease_state"] = normal_nacc_metadata["disease_state"].replace({
    "disease control": "normal control"
    })
# Compute descriptive stats
normal_nacc_metadata.groupby(["disease_state", "ethnicity"])["age"].describe()

,,count,mean,std,min,25%,50%,75%,max
disease_state,ethnicity,,,,,,,,
normal control,White,48.0,47.229167,11.255712,21.0,40.0,48.0,53.5,70.0


In [191]:
# Compute descriptive stats
sz_nacc_metadata.groupby(["disease_state", "ethnicity"])["age"].describe()

,,count,mean,std,min,25%,50%,75%,max
disease_state,ethnicity,,,,,,,,
schizophrenia,White,32.0,44.8125,10.5324,24.0,36.0,44.5,52.25,63.0


In [ ]:
print(normal_nacc_metadata["age"].describe())
print(normal_nacc_metadata["gender"].value_counts())

count    48.000000
mean     47.229167
std      11.255712
min      21.000000
25%      40.000000
50%      48.000000
75%      53.500000
max      70.000000
Name: age, dtype: float64
gender
male      37
female    11
Name: count, dtype: Int64


In [193]:
print(sz_nacc_metadata["age"].describe())
print(sz_nacc_metadata["gender"].value_counts())

count    32.0000
mean     44.8125
std      10.5324
min      24.0000
25%      36.0000
50%      44.5000
75%      52.2500
max      63.0000
Name: age, dtype: float64
gender
male      28
female     4
Name: count, dtype: Int64


In [ ]:
# Merge metadatas
metadata_nacc = pd.concat([normal_nacc_metadata, sz_nacc_metadata], axis=0)
metadata_nacc.head()
shape(metadata_nacc)

,sample_index,age_summary,disease_state,ethnicity,gender,tissue,disease,title,age,subject_id
0,85954,63 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,63,<NA>
1,85953,32 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,32,<NA>
2,220954,36 years,normal control,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,36,<NA>
3,220996,38 years,normal control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,38,<NA>
4,220958,58 years,normal control,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,58,<NA>
...,...,...,...,...,...,...,...,...,...,...
27,220963,62 years,schizophrenia,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,62,GSE202537_13017
28,85820,26 years,schizophrenia,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,26,GSE80655_X4301
29,220961,48 years,schizophrenia,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,48,GSE202537_1296
30,85909,40 years,schizophrenia,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,40,GSE80655_X3651


In [ ]:
# Sort sample_indexes
metadata_nacc = metadata_nacc.sort_values(by="sample_index")
metadata_nacc.head()

,sample_index,age_summary,disease_state,ethnicity,gender,tissue,disease,title,age,subject_id
0,85772,31 years,schizophrenia,White,female,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,31,GSE80655_X2353
30,85774,45 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,45,<NA>
22,85775,53 years,schizophrenia,White,female,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,53,GSE80655_X2976
45,85802,70 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,70,<NA>
16,85803,35 years,normal control,White,male,nucleus accumbens,mental disorder,RNA-sequencing of human post-mortem brain tissues,35,<NA>
...,...,...,...,...,...,...,...,...,...,...
27,221000,40 years,normal control,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,40,<NA>
11,221003,41 years,schizophrenia,White,female,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,41,GSE202537_843
21,221004,33 years,schizophrenia,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,33,GSE202537_878
25,221005,47 years,schizophrenia,White,male,nucleus accumbens,psychotic disorder,Diurnal alterations in gene expression across ...,47,GSE202537_930


In [198]:
# Save metadata to csv
metadata_nacc.to_csv('metadata_nacc_.csv', index=False)

### Prepare count martix

In [ ]:
# Filter exon_data for those samples present in metadata_nacc
# Get the unique sample indexes from metadata_nacc to use for query 
sample_idx = metadata_nacc['sample_index'].unique().tolist()
print(("Sample Indexes:", sample_idx)[0:5])
print(len(sample_idx))

Sample Indexes: [85772, 85774, 85775, 85802, 85803, 85804, 85805, 85814, 85819, 85820, 85823, 85825, 85828, 85836, 85837, 85883, 85884, 85886, 85907, 85908, 85909, 85925, 85941, 85942, 85943, 85953, 85954, 85956, 85966, 85987, 85988, 85998, 86002, 86033, 86039, 86044, 86045, 86046, 86047, 220937, 220940, 220942, 220944, 220946, 220949, 220950, 220952, 220954, 220958, 220961, 220962, 220963, 220964, 220966, 220967, 220968, 220970, 220971, 220972, 220973, 220976, 220977, 220978, 220979, 220981, 220982, 220983, 220984, 220985, 220986, 220988, 220989, 220993, 220996, 220998, 221000, 221003, 221004, 221005, 221007]
80


In [ ]:
# Convert the list to a comma-separated string
sample_idx_str = ",".join(map(str, sample_idx))
print((sample_idx_str)[0:5])

85772,85774,85775,85802,85803,85804,85805,85814,85819,85820,85823,85825,85828,85836,85837,85883,85884,85886,85907,85908,85909,85925,85941,85942,85943,85953,85954,85956,85966,85987,85988,85998,86002,86033,86039,86044,86045,86046,86047,220937,220940,220942,220944,220946,220949,220950,220952,220954,220958,220961,220962,220963,220964,220966,220967,220968,220970,220971,220972,220973,220976,220977,220978,220979,220981,220982,220983,220984,220985,220986,220988,220989,220993,220996,220998,221000,221003,221004,221005,221007


In [ ]:
sql = f"""
SELECT ed.sample_index, ed.exon_index, ed.read_count
FROM human_disease_b38_gc33.exon_data ed
WHERE ed.sample_index IN ({sample_idx_str})
LIMIT 5
"""
print(sql)
exon_data = datastore.query(sql)
display(exon_data)


SELECT ed.sample_index, ed.exon_index, ed.read_count
FROM human_disease_b38_gc33.exon_data ed
WHERE ed.sample_index IN (85772,85774,85775,85802,85803,85804,85805,85814,85819,85820,85823,85825,85828,85836,85837,85883,85884,85886,85907,85908,85909,85925,85941,85942,85943,85953,85954,85956,85966,85987,85988,85998,86002,86033,86039,86044,86045,86046,86047,220937,220940,220942,220944,220946,220949,220950,220952,220954,220958,220961,220962,220963,220964,220966,220967,220968,220970,220971,220972,220973,220976,220977,220978,220979,220981,220982,220983,220984,220985,220986,220988,220989,220993,220996,220998,221000,221003,221004,221005,221007)



,sample_index,exon_index,read_count
0,85772,12083,4.0
1,85774,12083,0.040408
2,85775,12083,1.1
3,85802,12083,1.06
4,85803,12083,1.06
...,...,...,...
32241915,86039,323018,205.4478
32241916,86044,323018,170.4055
32241917,86045,323018,114.5631
32241918,86046,323018,52.05908


In [18]:
# check if there are any duplicate exon indexes per sample index
print(exon_data["exon_index"].is_unique)
# check if there are any duplicate sample indexes
print(exon_data["sample_index"].is_unique)

# check if exon indexes are unique when grouped by sample index
print(len(exon_data.groupby("sample_index")["exon_index"].nunique()))
print(len(exon_data.groupby("sample_index")["exon_index"]))


False
False
80
80


In [22]:
exon_index_data = exon_data.sort_values(by="exon_index")
print(exon_index_data["exon_index"].unique())

<IntegerArray>
[     0,      1,      2,      3,      4,      5,      6,      7,      8,
      9,
 ...
 396435, 396436, 396437, 396438, 396439, 396440, 396441, 396442, 396443,
 396444]
Length: 396445, dtype: Int32


In [20]:
# Pivot table to make count matrix
# Pivot the long-format data to create a matrix (samples x exons)
count_matrix = exon_data.pivot_table(index='exon_index',
                                           columns='sample_index',
                                           values='read_count', 
                                           fill_value=0)
print(count_matrix.head())

sample_index  85772     85774     85775     85802     85803     85804   \
exon_index                                                               
0                1.0       1.0      0.74       1.0       1.3      0.92   
1              20.68   21.3498  18.64222      16.9      15.8  17.21556   
2              16.08  33.94348  16.77778     18.58     20.82  32.75003   
3              23.68  24.15652     23.54  24.15167  19.41837  24.72462   
4              27.64  25.75652     26.12  27.65167  28.69837  28.02462   

sample_index    85805     85814     85819     85820   ...    220988    220989  \
exon_index                                            ...                       
0                  0.0       0.0      0.26       0.0  ...  7.037838       0.0   
1             23.01188     22.94  12.65347  11.82829  ...  48.95189  109.8223   
2             29.51633     27.96     17.28  19.53171  ...  103.3423   131.424   
3             23.86512  21.32296     13.88      34.4  ...  68.04674  89.3231

In [171]:
count_matrix.index.name = None
count_matrix.columns.name = None

print(count_matrix)

# Save count matrix df to CSV file
count_matrix.to_csv("count_matrix_nacc.csv")


#### Extract exon_annotations

In [ ]:
sql ="""
SELECT 
    ea.exon_index,
    ea.exon_id,
    ea.chromosome,
    ea.start,
    ea."end" AS exon_end,
    ea.gene_id,
    ea.gene_name,
    ea.transcripts
FROM human_disease_b38_gc33.exon_annotation ea
WHERE ea.exon_index IN (
    SELECT DISTINCT ed.exon_index
    FROM human_disease_b38_gc33.exon_data ed
    JOIN human_disease_b38_gc33.samples s 
      ON ed.sample_index = s.sample_index
    WHERE s.disease_state IN ('schizophrenia', 'normal control', 'disease control', 'schizoaffective disorder')
    AND s.tissue = 'nucleus accumbens'
    AND s.ethnicity = 'White'
    LIMIT 5
)
"""

exon_annotation = datastore.query(sql)
display(exon_annotation)

Exception: Unexpected OPI access error: Could not create session: The proxy's IP address(212.237.134.41) is not in the accepted IP range

In [23]:
# Sanity check if exon indexes in exon_annotations match the merged count matrix
exon_annotation["exon_index"].nunique()

NameError: name 'exon_annotation' is not defined

In [ ]:
# sort indexes by exon_index
exon_annotation = exon_annotation.sort_values(by="exon_index")
exon_annotation.head()


,exon_index,exon_id,chromosome,start,exon_end,gene_id,gene_name,transcripts
8340,0,1:100000637-100000739,1,100000637,100000739,ENSG00000202259.1,RNU6-1318P,ENST00000365389.1
270634,1,1:100007034-100007156,1,100007034,100007156,ENSG00000117620.15,SLC35A3,ENST00000638371.1;ENST00000427993.7;ENST000005...
270635,1,1:100007034-100007156,1,100007034,100007156,ENSG00000283761.1,AC118553.2,ENST00000638792.1;ENST00000639037.1
137220,2,1:100011365-100011533,1,100011365,100011533,ENSG00000283761.1,AC118553.2,ENST00000638792.1;ENST00000639037.1
137219,2,1:100011365-100011533,1,100011365,100011533,ENSG00000117620.15,SLC35A3,ENST00000638371.1;ENST00000427993.7;ENST000005...
...,...,...,...,...,...,...,...,...
79888,396440,Y:9958419-9958529,Y,9958419,9958529,ENSG00000234950.1,RBMY2OP,ENST00000447105.1
334210,396441,Y:9958610-9958698,Y,9958610,9958698,ENSG00000234950.1,RBMY2OP,ENST00000447105.1
352553,396442,Y:9959112-9959423,Y,9959112,9959423,ENSG00000234950.1,RBMY2OP,ENST00000447105.1
389575,396443,Y:9989190-9989239,Y,9989190,9989239,ENSG00000236718.4,RBMY2QP,ENST00000651645.1


In [28]:
# Save to csv
exon_annotation.to_csv("exon_annotation_nacc.csv", index=False)

### Sanity checking the resulting data - get back to this

In [176]:
### Sanity checking data

# Checking data types and dimensions
print("Count matrix shape:", count_matrix.shape)
print("Row index (exon IDs) type:", count_matrix.index.dtype)
print("Column names (sample IDs) type:", count_matrix.columns.dtype)

# Inspect the first few rows and columns
print(count_matrix.head())

Count matrix shape: (396445, 80)
Row index (exon IDs) type: Int32
Column names (sample IDs) type: Int32
   85772     85774     85775     85802     85803     85804     85805   \
0     1.0       1.0      0.74       1.0       1.3      0.92       0.0   
1   20.68   21.3498  18.64222      16.9      15.8  17.21556  23.01188   
2   16.08  33.94348  16.77778     18.58     20.82  32.75003  29.51633   
3   23.68  24.15652     23.54  24.15167  19.41837  24.72462  23.86512   
4   27.64  25.75652     26.12  27.65167  28.69837  28.02462  26.30512   

     85814     85819     85820   ...    220988    220989    220993    220996  \
0       0.0      0.26       0.0  ...  7.037838       0.0       0.0  2.626667   
1     22.94  12.65347  11.82829  ...  48.95189  109.8223  0.826667     33.24   
2     27.96     17.28  19.53171  ...  103.3423   131.424  3.226667   66.1845   
3  21.32296     13.88      34.4  ...  68.04674  89.32318  1.573333  49.36272   
4  23.70296     13.88     36.46  ...  79.78007  112.9021 

In [178]:
# Get sample indexes from the count matrix columns and metadata
sample_idx_from_counts = set(count_matrix.columns.tolist())
sample_idx_from_metadata = set(metadata_nacc["sample_index"].tolist())

print("Number of sample_indexes in count matrix:", len(sample_idx_from_counts))
print("Number of sample_indexes in metadata:", len(sample_idx_from_metadata))

# Check if all sample IDs in the count matrix are in the metadata:
missing_in_metadata = sample_idx_from_counts.difference(sample_idx_from_metadata)
if missing_in_metadata:
    print("Sample indices missing in metadata:", missing_in_metadata)
else:
    print("All sample indexes in count matrix are present in metadata.")

Number of sample IDs in count matrix: 80
Number of sample IDs in metadata: 80
All sample IDs in count matrix are present in metadata.


In [179]:
# Check if exon indexes in count matrix are in exon_annotation
exon_idx_from_counts = set(count_matrix.index.tolist())
exon_idx_from_annotation = set(exon_annotation["exon_index"].tolist())
print("Number of exon_indexes in count matrix:", len(exon_idx_from_counts))
print("Number of exon_indexes in annotation:", len(exon_idx_from_annotation))
# Check if all exon indexes in the count matrix are in the annotation:
missing_in_annotation = exon_idx_from_counts.difference(exon_idx_from_annotation)
if missing_in_annotation:
    print("Exon indexes missing in annotation:", missing_in_annotation)
else:
    print("All exon indexes in count matrix are present in annotation.")
    

Number of exon_indexes in count matrix: 396445
Number of exon_indexes in annotation: 396445
All exon indexes in count matrix are present in annotation.


In [180]:
# Check number of samples in each disease condition
metadata_nacc["disease_state"].value_counts()

disease_state
normal control    48
schizophrenia     32
Name: count, dtype: Int64

In [7]:
# Group by title and check disease state c
metadata_nacc.groupby(["title", "disease_state"])["sample_index"].count()

NameError: name 'metadata_nacc' is not defined

In [181]:
# Check number of samples in each tissue
metadata_nacc["tissue"].value_counts()

tissue
nucleus accumbens    80
Name: count, dtype: Int64

In [ ]:
# There are duplicate exon indexes (rows) in exon_annotations (same exon is used by multiple genes)
dup_counts = exon_annotation['exon_index'].value_counts()
dup_counts[1:5]

exon_index
297498    22
297497    22
297336    15
297335    15
297504    14
          ..
396363     1
396374     1
396390     1
396427     1
125303     1
Name: count, Length: 396445, dtype: Int64

In [ ]:
# show the duplicated rows
duplicated_exons = exon_annotation[exon_annotation['exon_index'].isin(dup_counts[dup_counts > 1].index)]
duplicated_exons.head()

,exon_index,exon_id,chromosome,start,exon_end,gene_id,gene_name,transcripts
203,131129,15:89889049-89889140,15,89889049,89889140,ENSG00000157823.17,AP3S2,ENST00000336418.9;ENST00000558011.5;ENST000005...
204,131129,15:89889049-89889140,15,89889049,89889140,ENSG00000250021.7,ARPIN-AP3S2,ENST00000398333.7
292,133746,16:15118926-15119054,16,15118926,15119054,ENSG00000188599.17,NPIPP1,ENST00000534799.6
293,133746,16:15118926-15119054,16,15118926,15119054,ENSG00000270580.5,AC139256.3,ENST00000605794.5;ENST00000340301.5
304,134224,16:1662761-1662835,16,1662761,1662835,ENSG00000007545.16,CRAMP1,ENST00000397412.8;ENST00000293925.9
...,...,...,...,...,...,...,...,...
402706,386677,X:15664337-15664395,X,15664337,15664395,ENSG00000147003.7,CLTRN,ENST00000380342.4;ENST00000650271.1
402766,388474,X:38666121-38666309,X,38666121,38666309,ENSG00000250349.3,AF241726.2,ENST00000465127.1
402767,388474,X:38666121-38666309,X,38666121,38666309,ENSG00000156298.12,TSPAN7,ENST00000378482.6;ENST00000286824.6
402849,391206,X:55451495-55451582,X,55451495,55451582,MI0016906,hsa-mir-4536-1,t_MI0016906


In [184]:
# Sum counts per exon (sums all read count values across each exon)
exon_counts = count_matrix.sum(axis=1)
print("Distribution of exon total counts:")
print(exon_counts.describe())

# Optionally, filter out exons with total counts below a threshold (e.g., 10) - we get back to this later
threshold = 10
filtered_count_matrix_df = count_matrix.loc[exon_counts >= threshold]
print("Filtered count matrix shape:", filtered_count_matrix_df.shape)


Distribution of exon total counts:
count         396445.0
mean      23485.485747
std      721345.764226
min                0.0
25%         177.255931
50%         1812.43165
75%         8325.54946
max      220827033.636
dtype: Float64
Filtered count matrix shape: (374542, 80)


In [ ]:
# Check unique genes
unique_genes = exon_annotation['gene_name'].unique()

In [3]:
print(len(unique_genes))

print(len(exon_annotation['gene_id'].unique()))

60615
60699


In [ ]:
# Explore if any gene names or ID don't have a name or ID
ids_has_any_name = exon_annotation.groupby("gene_id", dropna=True)["gene_name"].apply(lambda s: s.notna().any())
ids_without_name = ids_has_any_name[ids_has_any_name == False].index.tolist()

# Names that never have a non-null gene_id anywhere
names_has_any_id = exon_annotation.groupby("gene_name", dropna=True)["gene_id"].apply(lambda s: s.notna().any())
names_without_id = names_has_any_id[names_has_any_id == False].index.tolist()

print("\nIDs with NO name at all:", len(ids_without_name))
print("Examples:", ids_without_name[:10])
print("Names with NO id at all:", len(names_without_id))
print("Examples:", names_without_id[:10])


IDs with NO name at all: 0
Examples: []
Names with NO id at all: 0
Examples: []


In [ ]:
# Check names that map to multiple IDs
name_to_ids_n = (exon_annotation.dropna(subset=["gene_id","gene_name"])
                   .groupby("gene_name")["gene_id"].nunique()
                   .sort_values(ascending=False))
names_multi_ids = name_to_ids_n[name_to_ids_n > 1]

# IDs that map to multiple Names
id_to_names_n = (exon_annotation.dropna(subset=["gene_id","gene_name"])
                   .groupby("gene_id")["gene_name"].nunique()
                   .sort_values(ascending=False))
ids_multi_names = id_to_names_n[id_to_names_n > 1]

print("\nGene names mapping to >1 gene_id:", len(names_multi_ids))
names_multi_ids.asDataFrame()
print(names_multi_ids)
print("\nGene IDs mapping to >1 gene_name:", len(ids_multi_names))
print(ids_multi_names.head(10))


Gene names mapping to >1 gene_id: 82
gene_name
hsa-mir-10401    4
AL732314.8       2
ASMTL            2
ASMTL-AS1        2
AL954722.1       2
                ..
AKAP17A          2
hsa-mir-1253     2
hsa-mir-423      2
hsa-mir-4477a    2
hsa-mir-4477b    2
Name: gene_id, Length: 82, dtype: int64

Gene IDs mapping to >1 gene_name: 0
Series([], Name: gene_name, dtype: int64)


In [ ]:
# Direct “orphan” rows: where one side is present but the other is null
orphan_id_rows = exon_annotation[exon_annotation["gene_id"].notna() & exon_annotation["gene_name"].isna()]
orphan_name_rows = exon_annotation[exon_annotation["gene_name"].notna() & exon_annotation["gene_id"].isna()]

print("\nRows with ID but no NAME:", orphan_id_rows.shape[0])
print(orphan_id_rows.head(5)[["gene_id","gene_name"]])
print("\nRows with NAME but no ID:", orphan_name_rows.shape[0])
print(orphan_name_rows.head(5)[["gene_id","gene_name"]])


Rows with ID but no NAME: 0
Empty DataFrame
Columns: [gene_id, gene_name]
Index: []

Rows with NAME but no ID: 0
Empty DataFrame
Columns: [gene_id, gene_name]
Index: []


If need to read data in

In [8]:
import pandas as pd
# read in metadata_nacc
metadata_nacc = pd.read_csv('metadata_nacc_.csv')

# Read in count matrix
count_matrix = pd.read_csv("count_matrix_nacc.csv")
print(count_matrix.index) 


# Import exon annotations
exon_annotation = pd.read_csv("input_data/combined_study/exon_annotation_nacc.csv", low_memory = False)

RuntimeError: CPU dispatcher tracer already initlized